In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
from IPython.display import clear_output
from operator import itemgetter

import time 

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NTS_SPENSER_matched_dir = r'' # use your path
NTS_SPENSER_matched_file = os.path.join(NTS_SPENSER_matched_dir, "")
df_NTS_SPENSER_matched = pd.read_csv(NTS_SPENSER_matched_file, index_col=None, header=0)

In [ ]:
len(df_NTS_SPENSER_matched)

In [ ]:
df_NTS_SPENSER_matched.head()

In [ ]:
#NTS_id_list = df_NTS_SPENSER_matched['IndividualID'].unique().tolist()

In [ ]:
NTS_day_id_list = df_NTS_SPENSER_matched['DayID'].unique().tolist()

In [ ]:
# Import the file with the days related to each individual
trips_dir = r'' # use your path


df_NTS_trips = pd.read_csv(
    trips_dir,
    sep='\t',
    usecols=['TripID',
             'DayID',               # ID given to all trips made by an individual on a given travel day - Created in SQL
             'IndividualID',        # Individual unique ID - Created in SQL
             'HouseholdID',         # Household unique ID - Created in SQL
             'PSUID',               # PSU unique ID - Created in SQL
             'JourSeq',             # Journey number on a given travel day
             'NumStages',           # Number of stages - actual number
             'MainMode_B04ID',      # Main mode of travel - publication table breakdown - 13 categories
             'TripPurpTo_B01ID',   # Trip purpose - full list - 23 categories
             'TripStartHours',      # Trip start time - hours component
             'TripStartMinutes',    # Trip start time - minutes component
             'TripStart',           # Trip start time - minutes past midnight
             'TripEndHours',        # Trip end time - hours component
             'TripEndMinutes',      # Trip end time - minutes component
             'TripEnd',             # Trip end time - minutes past midnight
             'TripDisIncSW',        # Trip distance - including short walk - miles - actual distance
             'TripTotalTime'        # Total trip time - minutes - actual time
             ]          
)
#persons_in.head()

In [ ]:
# Get the days that belong to the spenser agents
df_NTS_trips = df_NTS_trips.loc[(df_NTS_trips['DayID'].isin(NTS_day_id_list))]

In [ ]:
len(df_NTS_trips)

In [ ]:
## Update transport mode
mode_mapping = {
    1: 'walk',
     2: 'bike',
     3: 'car',  #'Car/van driver'
     4: 'car_passenger',  #'Car/van passenger'
     5: 'motorcycle',  
     6: 'car',  #'Other private transport',
     7: 'bus', #Bus in London',
     8: 'bus', #'Other local bus',
     9: 'bus', #'Non-local bus',
     10: 'metro', #'London Underground',
     11: 'train', #'Surface Rail',
     12: 'car',  #'Taxi/minicab',
     13: 'metro', #'Other public transport',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['MainMode_B04ID'] = df_NTS_trips['MainMode_B04ID'].map(mode_mapping)

In [ ]:
purp_mapping = {
    1: 'work',
     2: 'work',  #'In course of work',
     3: 'education',
     4: 'food_shop',  #'Food shopping',
     5: 'shop',  #'Non food shopping',
     6: 'medical', #'Personal business medical',
     7: 'eat',  #'Personal business eat/drink',
     8: 'other',  #'Personal business other',
     9: 'eat',  #'Eat/drink with friends',
     10: 'other',  #'Visit friends',
     11: 'leisure_act',  #'Other social',
     12: 'leisure_act',  #'Entertain/ public activity',
     13: 'leisure_sport',  #'Sport: participate',
     14: 'home',  #'Holiday: base',
     15: 'leisure_sport',  #'Day trip/just walk',
     16: 'leisure_act',  #'Other non-escort',
     17: 'escort_home',  #'Escort home',
     18: 'escort_work',  #'Escort work',
     19: 'escort_work',  #'Escort in course of work',
     20: 'escort_education',  #'Escort education',
     21: 'escort_leisure',  #'Escort shopping/personal business',
     22: 'escort_other',  #'Other escort',
     23: 'home',  #'Home',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['TripPurpTo_B01ID'] = df_NTS_trips['TripPurpTo_B01ID'].map(purp_mapping)

In [ ]:
df_NTS_trips.sort_values('TripID').head(10)

In [ ]:
df_NTS_trips = df_NTS_trips.where(df_NTS_trips.notnull(), None)

In [ ]:
# LIMITATION 1: ALL SPENSER agents start first trip from home
## We need to remove those first trips which purpose = home

### Select those trips which JourSeq =  1 and pusprose = home
df_NTS_trips_JourSeq_1_home = df_NTS_trips.loc[(df_NTS_trips['JourSeq'] == 1) & (df_NTS_trips['TripPurpTo_B01ID'] == 'home')]

In [ ]:
df_NTS_trips_JourSeq_1_home

In [ ]:
df_NTS_trips.loc[(df_NTS_trips['DayID'] ==2016069403)].sort_values('JourSeq')

In [ ]:
len(df_NTS_trips_JourSeq_1_home)

In [ ]:
# Get the tripID that belong to those trips
NTS_trips_JourSeq_1_home_tripID_list = df_NTS_trips_JourSeq_1_home['TripID'].unique().tolist()

In [ ]:
# Remove those trips from the main dataframe 'df_NTS_trips'
df_NTS_trips = df_NTS_trips.loc[~(df_NTS_trips['TripID'].isin(NTS_trips_JourSeq_1_home_tripID_list))]

In [ ]:
len(df_NTS_trips)

In [ ]:
##############################################################
##############################################################
#



##############################################################
##############################################################

In [ ]:
# LIMITATION 2: ALL SPENSER agents last trip ends at home unless first trip purpose = last trip purpose
## We need to add a last trip to those which last trip which purpose != home



In [ ]:
# Spilts persons Dataframe into MSOA areas (whole households are stored in OA areas)
grps_trips_per_day_split = df_NTS_trips.groupby('DayID')

In [ ]:
df_NTS_trips

In [ ]:
total_days = len(grps_trips_per_day_split)

In [ ]:
#example
#grps_trips_per_day_split.get_group(2016041752)

In [ ]:
last_trip_to_home_added_path = r''

In [ ]:
counter = 0

counter_day = 0


total_days = len(grps_trips_per_day_split)

for day_name, trip_grp in grps_trips_per_day_split:
    
    counter_day += 1
    
    clear_output(wait=True)
    print(f'Days processed: {counter_day} of {total_days}')

    trip_grp = trip_grp.sort_values('JourSeq', ascending=True)
    last_trip = trip_grp.iloc[-1:]
    last_trip_purpose = trip_grp.iloc[-1, 8]
    
    
    
    if (last_trip_purpose != 'home'):
        
        
        #print(trip_grp)
        
        counter += 1
        temp_list = []
        
        # Same ID's
        TripID = last_trip.iloc[-1, 0]
        DayID = last_trip.iloc[-1, 1]
        IndividualID = last_trip.iloc[-1, 2]
        HouseholdID = last_trip.iloc[-1, 3]
        PSUID = last_trip.iloc[-1, 4]
        
        # Get the distance from previous
        TripDisIncSW = last_trip.iloc[-1, 15]
        
        # Purpose home
        TripPurpTo_B01ID = 'home'
        
        # Same transport mode
        MainMode_B04ID = last_trip.iloc[-1, 7]
        
        #Journey sequence = last trip + 1
        JourSeq  = int(last_trip.iloc[-1, 5]) + 1
        NumStages = 1

    
        # Append data into the temp_list
        temp_list.append(TripID)
        temp_list.append(DayID)
        temp_list.append(IndividualID)
        temp_list.append(HouseholdID)
        temp_list.append(PSUID)
        temp_list.append(TripPurpTo_B01ID)
        temp_list.append(MainMode_B04ID)
        temp_list.append(JourSeq)
        temp_list.append(NumStages)
        temp_list.append(TripDisIncSW)
        #temp_list.append(TripStartHours)
        #temp_list.append(TripStartMinutes)
        #temp_list.append(TripStart)

        
        # Create an empty dataframe
        df_last_trip_to_home_added = pd.DataFrame([temp_list], columns=['TripID',
                                                                        'DayID',
                                                                        'IndividualID',
                                                                        'HouseholdID',
                                                                        'PSUID',
                                                                        'TripPurpTo_B01ID',
                                                                        'MainMode_B04ID',
                                                                        'JourSeq',
                                                                        'NumStages',
                                                                        'TripDisIncSW'
                                                                       ])
        
        
        #print(df_last_trip_to_home_added)
        
        #sys.exit()
        
        if (counter == 1):
        
            # save the data in a csv file
            df_last_trip_to_home_added.to_csv(last_trip_to_home_added_path, encoding='utf-8', mode='a', index=False, header=True)

        else:
            
            # save the data in a csv file
            df_last_trip_to_home_added.to_csv(last_trip_to_home_added_path, encoding='utf-8', mode='a', index=False, header=False)

            
print('The code has finished. Check the results, amigo!')

In [ ]:
# Read the last_trip_to_home_added

last_trip_to_home_added_dir = r'' # use your path
last_trip_to_home_added_file = os.path.join(last_trip_to_home_added_dir, "last_trip_home_added.csv")
df_last_trip_to_home_added = pd.read_csv(last_trip_to_home_added_file, index_col=None, header=0)

In [ ]:
len(df_last_trip_to_home_added)

In [ ]:
# Concatenate previous trips with the new ones

In [ ]:
frames = [df_NTS_trips, df_last_trip_to_home_added]

df_NTS_trips_all = pd.concat(frames)

In [ ]:
len(df_NTS_trips_all)

In [ ]:
len(df_NTS_trips) + len(df_last_trip_to_home_added)

In [ ]:
df_NTS_trips_all

In [ ]:
all_trip_to_concatenated_path = r''

In [ ]:
df_NTS_trips_all.to_csv(all_trip_to_concatenated_path, encoding='utf-8', mode='a', index=False, header=True)
